In [1]:
# @title Imports

import dataclasses
import datetime
import functools
import math
import re
from typing import Optional

import cartopy.crs as ccrs
#from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import checkpoint
from graphcast import data_utils
from graphcast import graphcast
from graphcast import normalization
from graphcast import rollout
from graphcast import xarray_jax
from graphcast import xarray_tree
from IPython.display import HTML
import ipywidgets as widgets
import haiku as hk
import jax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import xarray




def parse_file_parts(file_name):
  return dict(part.split("-", 1) for part in file_name.split("_"))


In [2]:
# @title Choose the model
# Rewrite by S.F. Sune, https://github.com/sfsun67.
'''
    We have three options. Acquiring from https://console.cloud.google.com/storage/browser/dm_graphcast:
    GraphCast - ERA5 1979-2017 - resolution 0.25 - pressure levels 37 - mesh 2to6 - precipitation input and output.npz
    GraphCast_operational - ERA5-HRES 1979-2021 - resolution 0.25 - pressure levels 13 - mesh 2to6 - precipitation output only.npz
    GraphCast_small - ERA5 1979-2015 - resolution 1.0 - pressure levels 13 - mesh 2to5 - precipitation input and output.npz
'''
# find the result in this path /root/data/params, and list of names of all files in the "params/", with the "params/" perfix removed from the names.
import os
import glob

# Define the directory path
dir_path_params = "/root/data/params"

# Use glob to get all file paths in the directory
file_paths_params = glob.glob(os.path.join(dir_path_params, "*"))

# Remove the directory path and the ".../params/" prefix from each file name
params_file_options = [os.path.basename(path) for path in file_paths_params]


random_mesh_size = widgets.IntSlider(
    value=4, min=4, max=6, description="Mesh size:")
random_gnn_msg_steps = widgets.IntSlider(
    value=4, min=1, max=32, description="GNN message steps:")
random_latent_size = widgets.Dropdown(
    options=[int(2**i) for i in range(4, 10)], value=32,description="Latent size:")
random_levels = widgets.Dropdown(
    options=[13, 37], value=13, description="Pressure levels:")


params_file = widgets.Dropdown(
    options=params_file_options,
    description="Params file:",
    layout={"width": "max-content"})

source_tab = widgets.Tab([
    widgets.VBox([
        random_mesh_size,
        random_gnn_msg_steps,
        random_latent_size,
        random_levels,
    ]),
    params_file,
])
source_tab.set_title(0, "Random")
source_tab.set_title(1, "Checkpoint")
widgets.VBox([
    source_tab,
    widgets.Label(value="Run the next cell to load the model. Rerunning this cell clears your selection.")
])


In [3]:
# @title Load the model

source = source_tab.get_title(source_tab.selected_index)

if source == "Random":
  params = None  # Filled in below
  state = {}
  model_config = graphcast.ModelConfig(
      resolution=0,
      mesh_size=random_mesh_size.value,
      latent_size=random_latent_size.value,
      gnn_msg_steps=random_gnn_msg_steps.value,
      hidden_layers=1,
      radius_query_fraction_edge_length=0.6)
  task_config = graphcast.TaskConfig(
      input_variables=graphcast.TASK.input_variables,
      target_variables=graphcast.TASK.target_variables,
      forcing_variables=graphcast.TASK.forcing_variables,
      pressure_levels=graphcast.PRESSURE_LEVELS[random_levels.value],
      input_duration=graphcast.TASK.input_duration,
  )
else:
  assert source == "Checkpoint"
  '''with gcs_bucket.blob(f"params/{params_file.value}").open("rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)'''
  
  with open(f"{dir_path_params}/{params_file.value}", "rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)
  
  params = ckpt.params
  state = {}

  model_config = ckpt.model_config
  task_config = ckpt.task_config
  print("Model description:\n", ckpt.description, "\n")
  print("Model license:\n", ckpt.license, "\n")

model_config

ModelConfig(resolution=0, mesh_size=4, latent_size=32, gnn_msg_steps=4, hidden_layers=1, radius_query_fraction_edge_length=0.6, mesh2grid_edge_normalization_factor=None)

In [4]:
# @title Get and filter the list of available example datasets
# Rewrite by S.F. Sune, https://github.com/sfsun67.
# find the result in this path /root/data/dataset, and list of names of all files in the "dataset/", with the "dataset/" perfix removed from the names.

# Define the directory path
dir_path_dataset = "/root/data/dataset"

# Use glob to get all file paths in the directory
file_paths_dataset = glob.glob(os.path.join(dir_path_dataset, "*"))

# Remove the directory path and the ".../params/" prefix from each file name
dataset_file_options = [os.path.basename(path) for path in file_paths_dataset]
#print("dataset_file_options: ", dataset_file_options)

# Remove "dataset-" prefix from each file name
dataset_file_options = [name.removeprefix("dataset-") for name in dataset_file_options]


def data_valid_for_model(
    file_name: str, model_config: graphcast.ModelConfig, task_config: graphcast.TaskConfig):
  file_parts = parse_file_parts(file_name.removesuffix(".nc"))
  #print("file_parts: ", file_parts)
  return (
      model_config.resolution in (0, float(file_parts["res"])) and
      len(task_config.pressure_levels) == int(file_parts["levels"]) and
      (
          ("total_precipitation_6hr" in task_config.input_variables and
           file_parts["source"] in ("era5", "fake")) or
          ("total_precipitation_6hr" not in task_config.input_variables and
           file_parts["source"] in ("hres", "fake"))
      )
  )


dataset_file = widgets.Dropdown(
    options=[
        (", ".join([f"{k}: {v}" for k, v in parse_file_parts(option.removesuffix(".nc")).items()]), option)
        for option in dataset_file_options
        if data_valid_for_model(option, model_config, task_config)
    ],
    description="Dataset file:",
    layout={"width": "max-content"})
widgets.VBox([
    dataset_file,
    widgets.Label(value="Run the next cell to load the dataset. Rerunning this cell clears your selection and refilters the datasets that match your model.")
])

In [5]:
# @title Load weather data

if not data_valid_for_model(dataset_file.value, model_config, task_config):
  raise ValueError(
      "Invalid dataset file, rerun the cell above and choose a valid dataset file.")

'''with gcs_bucket.blob(f"dataset/{dataset_file.value}").open("rb") as f:
  example_batch = xarray.load_dataset(f).compute()'''

with open(f"{dir_path_dataset}/dataset-{dataset_file.value}", "rb") as f:
  example_batch = xarray.load_dataset(f).compute()

assert example_batch.dims["time"] >= 3  # 2 for input, >=1 for targets

print(", ".join([f"{k}: {v}" for k, v in parse_file_parts(dataset_file.value.removesuffix(".nc")).items()]))

example_batch

source: era5, date: 2022-01-01, res: 1.0, levels: 13, steps: 12


<xarray.Dataset>
Dimensions:                       (lon: 360, lat: 181, time: 14, level: 13,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * time                          (time) timedelta64[ns] 0 days 00:00:00 ... ...
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
    datetime                      (batch, time) datetime64[ns] 2022-01-01 ......
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 250.7 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.931e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon) float32 -0.4393 ......
    10m_u_component_of_wind       (batch, time, lat, lon) float32 1.309 ... -...
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 238....
    geopotential                  (batch, time, level, lat, lon) float32 1.98...
    u_component_of_wind           (batch, time, level, lat, lon) float32 -0.0...
    v_component_of_wind           (batch, time, level, lat, lon) float32 0.00...
    vertical_velocity             (batch, time, level, lat, lon) float32 -0.0...
    specific_humidity             (batch, time, level, lat, lon) float32 2.89...

In [16]:
print("example_batch", example_batch)

example_batch <xarray.Dataset>
Dimensions:                       (lon: 360, lat: 181, time: 14, level: 13,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * time                          (time) timedelta64[ns] 0 days 00:00:00 ... ...
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
    datetime                      (batch, time) datetime64[ns] 2022-01-01 ......
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 250.7 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.931e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon

In [17]:
example_batch_1_7 = example_batch.isel(time=slice(2, 12))
example_batch_1_7

<xarray.Dataset>
Dimensions:                       (lon: 360, lat: 181, time: 10, level: 13,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * time                          (time) timedelta64[ns] 0 days 12:00:00 ... ...
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
    datetime                      (batch, time) datetime64[ns] 2022-01-01T12:...
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 248.3 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.983e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon) float32 -0.04184 .....
    10m_u_component_of_wind       (batch, time, lat, lon) float32 0.04857 ......
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 238....
    geopotential                  (batch, time, level, lat, lon) float32 1.98...
    u_component_of_wind           (batch, time, level, lat, lon) float32 0.00...
    v_component_of_wind           (batch, time, level, lat, lon) float32 0.00...
    vertical_velocity             (batch, time, level, lat, lon) float32 0.00...
    specific_humidity             (batch, time, level, lat, lon) float32 2.89...

In [24]:
# @title Choose training and eval data to extract
train_steps = widgets.IntSlider(
    value=1, min=1, max=example_batch.sizes["time"]-2, description="Train steps")
eval_steps = widgets.IntSlider(
    value=example_batch.sizes["time"]-2, min=1, max=example_batch.sizes["time"]-2, description="Eval steps")

widgets.VBox([
    train_steps,
    eval_steps,
    widgets.Label(value="Run the next cell to extract the data. Rerunning this cell clears your selection.")
])

In [25]:
# @title Extract training and eval data

train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
    **dataclasses.asdict(task_config))

eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{eval_steps.value*6}h"),
    **dataclasses.asdict(task_config))

print("All Examples:  ", example_batch.dims.mapping)
print("Train Inputs:  ", train_inputs.dims.mapping)
print("Train Targets: ", train_targets.dims.mapping)
print("Train Forcings:", train_forcings.dims.mapping)
print("Eval Inputs:   ", eval_inputs.dims.mapping)
print("Eval Targets:  ", eval_targets.dims.mapping)
print("Eval Forcings: ", eval_forcings.dims.mapping)


All Examples:   {'lon': 360, 'lat': 181, 'time': 14, 'level': 13, 'batch': 1}
Train Inputs:   {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Train Targets:  {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Train Forcings: {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360}
Eval Inputs:    {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Eval Targets:   {'batch': 1, 'time': 12, 'lat': 181, 'lon': 360, 'level': 13}
Eval Forcings:  {'batch': 1, 'time': 12, 'lat': 181, 'lon': 360}


In [36]:
example_batch_0 = example_batch.isel(time=slice(0, 4))
example_batch_1 = example_batch.isel(time=slice(1, 5))
# ...
example_batch_1 = example_batch.isel(time=slice(7, 11))


In [37]:
# @title Extract training and eval data

train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch_1, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
    **dataclasses.asdict(task_config))

eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{eval_steps.value*6}h"),
    **dataclasses.asdict(task_config))

print("All Examples:  ", example_batch.dims.mapping)
print("Train Inputs:  ", train_inputs.dims.mapping)
print("Train Targets: ", train_targets.dims.mapping)
print("Train Forcings:", train_forcings.dims.mapping)
print("Eval Inputs:   ", eval_inputs.dims.mapping)
print("Eval Targets:  ", eval_targets.dims.mapping)
print("Eval Forcings: ", eval_forcings.dims.mapping)


All Examples:   {'lon': 360, 'lat': 181, 'time': 14, 'level': 13, 'batch': 1}
Train Inputs:   {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Train Targets:  {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Train Forcings: {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360}
Eval Inputs:    {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Eval Targets:   {'batch': 1, 'time': 12, 'lat': 181, 'lon': 360, 'level': 13}
Eval Forcings:  {'batch': 1, 'time': 12, 'lat': 181, 'lon': 360}


In [38]:
train_inputs

<xarray.Dataset>
Dimensions:                       (batch: 1, time: 2, lat: 181, lon: 360,
                                   level: 13)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * time                          (time) timedelta64[ns] -1 days +18:00:00 00...
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
Dimensions without coordinates: batch
Data variables: (12/18)
    2m_temperature                (batch, time, lat, lon) float32 247.3 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.969e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon) float32 -0.02237 .....
    10m_u_component_of_wind       (batch, time, lat, lon) float32 0.179 ... -...
    total_precipitation_6hr       (batch, time, lat, lon) float32 0.0 ... 5.9...
    temperature                   (batch, time, level, lat, lon) float32 238....
    ...                            ...
    year_progress_sin             (batch, time) float32 0.03708 0.04138
    year_progress_cos             (batch, time) float32 0.9993 0.9991
    day_progress_sin              (batch, time, lon) float32 -1.0 ... -0.01745
    day_progress_cos              (batch, time, lon) float32 1.192e-08 ... 0....
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0

In [39]:
train_targets

<xarray.Dataset>
Dimensions:                  (batch: 1, time: 2, lat: 181, lon: 360, level: 13)
Coordinates:
  * lon                      (lon) float32 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * lat                      (lat) float32 -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * time                     (time) timedelta64[ns] 06:00:00 12:00:00
  * level                    (level) int32 50 100 150 200 ... 700 850 925 1000
Dimensions without coordinates: batch
Data variables:
    2m_temperature           (batch, time, lat, lon) float32 249.6 ... 245.6
    mean_sea_level_pressure  (batch, time, lat, lon) float32 9.94e+04 ... 1.0...
    10m_v_component_of_wind  (batch, time, lat, lon) float32 0.1735 ... -0.4024
    10m_u_component_of_wind  (batch, time, lat, lon) float32 0.7688 ... -0.2829
    total_precipitation_6hr  (batch, time, lat, lon) float32 7.825e-05 ... 3....
    temperature              (batch, time, level, lat, lon) float32 239.6 ......
    geopotential             (batch, time, level, lat, lon) float32 1.993e+05...
    u_component_of_wind      (batch, time, level, lat, lon) float32 0.0008698...
    v_component_of_wind      (batch, time, level, lat, lon) float32 -0.001092...
    vertical_velocity        (batch, time, level, lat, lon) float32 -0.007226...
    specific_humidity        (batch, time, level, lat, lon) float32 2.923e-06...

In [12]:
# @title Load normalization data
# Rewrite by S.F. Sune, https://github.com/sfsun67.
dir_path_stats = "/root/data/stats"

with open(f"{dir_path_stats}/stats-diffs_stddev_by_level.nc", "rb") as f:
  diffs_stddev_by_level = xarray.load_dataset(f).compute()
with open(f"{dir_path_stats}/stats-mean_by_level.nc", "rb") as f:
  mean_by_level = xarray.load_dataset(f).compute()
with open(f"{dir_path_stats}/stats-stddev_by_level.nc", "rb") as f:
  stddev_by_level = xarray.load_dataset(f).compute()

In [13]:
# @title Build jitted functions, and possibly initialize random weights
# Construct the model and initialize the weights.
# 构建模型并初始化权重

# 模型组网
# Construct the model
def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """Constructs and wraps the GraphCast Predictor."""
  # Deeper one-step predictor.
  predictor = graphcast.GraphCast(model_config, task_config)

  # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
  # from/to float32 to/from BFloat16.
  predictor = casting.Bfloat16Cast(predictor)

  # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
  # BFloat16 happens after applying normalization to the inputs/targets.
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)

  # Wraps everything so the one-step model can produce trajectories.
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor

# 前向运算
# forward
@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  return predictor(inputs, targets_template=targets_template, forcings=forcings)

# 计算损失函数
# loss function
@hk.transform_with_state    # used to convert a pure function into a stateful function
def loss_fn(model_config, task_config, inputs, targets, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)    # constructs and wraps a GraphCast Predictor, which is a model used for making predictions in a graph-based machine learning task.
  loss, diagnostics = predictor.loss(inputs, targets, forcings)
  return xarray_tree.map_structure(
      lambda x: xarray_jax.unwrap_data(x.mean(), require_jax=True),
      (loss, diagnostics))

# 计算梯度
# gradient
def grads_fn(params, state, model_config, task_config, inputs, targets, forcings):
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

# Jax doesn't seem to like passing configs as args through the jit. Passing it
# in via partial (instead of capture by closure) forces jax to invalidate the
# jit cache if you change configs.
def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

# Always pass params and state, so the usage below are simpler
def with_params(fn):
  return functools.partial(fn, params=params, state=state)

# Our models aren't stateful, so the state is always empty, so just return the
# predictions. This is requiredy by our rollout code, and generally simpler.
def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

init_jitted = jax.jit(with_configs(run_forward.init))

if params is None:
  params, state = init_jitted(
      rng=jax.random.PRNGKey(0),
      inputs=train_inputs,
      targets_template=train_targets,
      forcings=train_forcings)

loss_fn_jitted = drop_state(with_params(jax.jit(with_configs(loss_fn.apply))))
grads_fn_jitted = with_params(jax.jit(with_configs(grads_fn)))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(
    run_forward.apply))))

# Run the model

Note that the cell below may take a while (possibly minutes) to run the first time you execute them, because this will include the time it takes for the code to compile. The second time running will be significantly faster.

This use the python loop to iterate over prediction steps, where the 1-step prediction is jitted. This has lower memory requirements than the training steps below, and should enable making prediction with the small GraphCast model on 1 deg resolution data for 4 steps.

In [14]:
# @title Autoregressive rollout (loop in python)

assert model_config.resolution in (0, 360. / eval_inputs.sizes["lon"]), (
  "Model resolution doesn't match the data resolution. You likely want to "
  "re-filter the dataset list, and download the correct data.")

print("Inputs:  ", eval_inputs.dims.mapping)
print("Targets: ", eval_targets.dims.mapping)
print("Forcings:", eval_forcings.dims.mapping)

predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings)
predictions

Inputs:   {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Targets:  {'batch': 1, 'time': 1, 'lat': 181, 'lon': 360, 'level': 13}
Forcings: {'batch': 1, 'time': 1, 'lat': 181, 'lon': 360}


ValueError: 'grid2mesh_gnn/~_networks_builder/encoder_nodes_grid_nodes_mlp/~/linear_0/w' with retrieved shape (98, 32) does not match shape=[186, 32] dtype=dtype(bfloat16)

# Train the model

The following operations require a large amount of memory and, depending on the accelerator being used, will only fit the very small "random" model on low resolution data. It uses the number of training steps selected above.

The first time executing the cell takes more time, as it include the time to jit the function.

In [ ]:
# @title Loss computation (autoregressive loss over multiple steps)
loss, diagnostics = loss_fn_jitted(
    rng=jax.random.PRNGKey(0),
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)

print("Loss:", float(loss))

Loss: 14.4375


In [ ]:
# @title Gradient computation (backprop through time)
loss, diagnostics, next_state, grads = grads_fn_jitted(
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)
mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")

Loss: 109.3542, Mean |grad|: 2.267955


In [ ]:
# @title Autoregressive rollout (keep the loop in JAX)
print("Inputs:  ", train_inputs.dims.mapping)
print("Targets: ", train_targets.dims.mapping)
print("Forcings:", train_forcings.dims.mapping)

predictions = run_forward_jitted(
    rng=jax.random.PRNGKey(0),
    inputs=train_inputs,
    targets_template=train_targets * np.nan,
    forcings=train_forcings)
predictions

Inputs:   {'batch': 1, 'time': 2, 'lat': 181, 'lon': 360, 'level': 13}
Targets:  {'batch': 1, 'time': 1, 'lat': 181, 'lon': 360, 'level': 13}
Forcings: {'batch': 1, 'time': 1, 'lat': 181, 'lon': 360}


<xarray.Dataset>
Dimensions:                  (time: 1, batch: 1, lat: 181, lon: 360, level: 13)
Coordinates:
  * time                     (time) timedelta64[ns] 06:00:00
  * lon                      (lon) float32 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * lat                      (lat) float32 -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
  * level                    (level) int32 50 100 150 200 ... 700 850 925 1000
Dimensions without coordinates: batch
Data variables:
    10m_u_component_of_wind  (time, batch, lat, lon) float32 xarray_jax.JaxAr...
    10m_v_component_of_wind  (time, batch, lat, lon) float32 xarray_jax.JaxAr...
    2m_temperature           (time, batch, lat, lon) float32 xarray_jax.JaxAr...
    geopotential             (time, batch, level, lat, lon) float32 xarray_ja...
    mean_sea_level_pressure  (time, batch, lat, lon) float32 xarray_jax.JaxAr...
    specific_humidity        (time, batch, level, lat, lon) float32 xarray_ja...
    temperature              (time, batch, level, lat, lon) float32 xarray_ja...
    total_precipitation_6hr  (time, batch, lat, lon) float32 xarray_jax.JaxAr...
    u_component_of_wind      (time, batch, level, lat, lon) float32 xarray_ja...
    v_component_of_wind      (time, batch, level, lat, lon) float32 xarray_ja...
    vertical_velocity        (time, batch, level, lat, lon) float32 xarray_ja...

In [41]:
# 对数据集进行迭代
for i in range(12):
    example_batch_slice = example_batch.isel(time=slice(i, 4+i))
    train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
        example_batch_slice, target_lead_times=slice("6h", f"{train_steps.value*6}h"),
        **dataclasses.asdict(task_config))
    # @title Gradient computation (backprop through time)
    loss, diagnostics, next_state, grads = grads_fn_jitted(
        inputs=train_inputs,
        targets=train_targets,
        forcings=train_forcings)
    mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
    print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")

# @title Gradient computation (backprop through time)
loss, diagnostics, next_state, grads = grads_fn_jitted(
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)
mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")

NameError: name 'grads_fn_jitted' is not defined